In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import warnings
import psycopg2
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
import sys
import pickle
from pprint import pprint

sys.path.insert(0, os.path.realpath('..\src'))
from database.database_config import DB_NAME, DB_USER, DB_PASSWORD, DB_HOST

In [2]:
# shooting_merge = pd.read_pickle(os.path.realpath('../data/preprocessed/shooting_merge.pkl'))
# possession_merge = pd.read_pickle(os.path.realpath('../data/preprocessed/possession_merge.pkl'))
# scores_overall = pd.read_pickle(os.path.realpath('../data/preprocessed/scores_overall.pkl'))
# scores_by_team = pd.read_pickle(os.path.realpath('../data/preprocessed/scores_by_team_merge.pkl'))

In [3]:
shooting_merge = pd.read_csv(os.path.realpath('../data/preprocessed/shooting_merge.csv'))
possession_merge = pd.read_csv(os.path.realpath('../data/preprocessed/possession_merge.csv'))
scores_overall = pd.read_csv(os.path.realpath('../data/preprocessed/scores_overall.csv'))
scores_by_team = pd.read_csv(os.path.realpath('../data/preprocessed/scores_by_team_merge.csv'))

In [4]:
scores_overall.head()

,_week_,_day_,_date_,_time_,home,score,away,attendance,venue,referee,xg_home,xg_away,homegoal,awaygoal
0,1,Fri,2017-08-18,20:15:00,Leganés,1–0,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,1.4,1.2,1,0
1,1,Fri,2017-08-18,22:15:00,Valencia,1–0,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,1.9,0.3,1,0
2,1,Sat,2017-08-19,18:15:00,Celta Vigo,2–3,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,2.4,2,3
3,1,Sat,2017-08-19,20:15:00,Girona,2–2,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,2.1,0.8,2,2
4,1,Sat,2017-08-19,22:15:00,Sevilla,1–1,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,1.1,1,1


In [5]:
scores_by_team.head()

,_date_,_time_,comp,round,_day_,attendance,referee,home,away,homegoal,xg_home,awaygoal,xg_away,homeformation,awayformation,homepossession,awaypossession,homecaptain,awaycaptain
0,2017-08-18,20:15:00,La Liga,Matchweek 1,Fri,9231.0,José Munuera,Leganés,Alavés,1,1.4,0,1.2,4-2-3-1,4-4-2,54.0,46.0,Martín Mantovani,Manu García
1,2017-08-18,22:15:00,La Liga,Matchweek 1,Fri,35971.0,Jesús Gil,Valencia,Las Palmas,1,1.9,0,0.3,4-4-2,4-5-1,48.0,52.0,Daniel Parejo,Jonathan Viera
2,2017-08-19,22:15:00,La Liga,Matchweek 1,Sat,30487.0,Alejandro Hernández,Sevilla,Espanyol,1,2.1,1,1.1,4-1-4-1,4-4-2,62.0,38.0,Jesús Navas,Gerard Moreno
3,2017-08-19,20:15:00,La Liga,Matchweek 1,Sat,11511.0,Juan Martínez,Girona,Atlético Madrid,2,2.1,2,0.8,3-4-3,4-4-2,53.0,47.0,Álex Granell,Gabi
4,2017-08-19,18:15:00,La Liga,Matchweek 1,Sat,16961.0,Antonio Matéu Lahoz,Celta Vigo,Real Sociedad,2,1.6,3,2.4,4-3-3,4-3-3,53.0,47.0,Hugo Mallo,Xabi Prieto


In [6]:
scores_merge_table = scores_overall.merge(scores_by_team, on = ['_date_', '_time_', '_day_', 'home', 'away', 'attendance', 'homegoal', 'awaygoal', 'xg_home', 'xg_away', 'referee'])
scores_merge_table

,_week_,_day_,_date_,_time_,home,score,away,attendance,venue,referee,...,homegoal,awaygoal,comp,round,homeformation,awayformation,homepossession,awaypossession,homecaptain,awaycaptain
0,1,Fri,2017-08-18,20:15:00,Leganés,1–0,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,...,1,0,La Liga,Matchweek 1,4-2-3-1,4-4-2,54.0,46.0,Martín Mantovani,Manu García
1,1,Fri,2017-08-18,22:15:00,Valencia,1–0,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,...,1,0,La Liga,Matchweek 1,4-4-2,4-5-1,48.0,52.0,Daniel Parejo,Jonathan Viera
2,1,Sat,2017-08-19,18:15:00,Celta Vigo,2–3,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,...,2,3,La Liga,Matchweek 1,4-3-3,4-3-3,53.0,47.0,Hugo Mallo,Xabi Prieto
3,1,Sat,2017-08-19,20:15:00,Girona,2–2,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,...,2,2,La Liga,Matchweek 1,3-4-3,4-4-2,53.0,47.0,Álex Granell,Gabi
4,1,Sat,2017-08-19,22:15:00,Sevilla,1–1,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,...,1,1,La Liga,Matchweek 1,4-1-4-1,4-4-2,62.0,38.0,Jesús Navas,Gerard Moreno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,38,Sun,2022-05-22,22:00:00,Sevilla,1–0,Athletic Club,31305.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,...,1,0,La Liga,Matchweek 38,4-3-3,4-4-2,65.0,35.0,Jesús Navas,Iker Muniain
1896,38,Sun,2022-05-22,22:00:00,Real Sociedad,1–2,Atlético Madrid,23586.0,Estadio Municipal de Anoeta,Jesús Gil,...,1,2,La Liga,Matchweek 38,4-4-2◆,3-5-2,61.0,39.0,Asier Illarramendi,Koke
1897,38,Sun,2022-05-22,20:00:00,Granada,0–0,Espanyol,17951.0,Estadio Nuevo Los Cármenes,Alejandro Hernández,...,0,0,La Liga,Matchweek 38,4-3-3,4-4-2,40.0,60.0,Víctor Díaz,David López
1898,38,Sun,2022-05-22,17:30:00,Elche,3–1,Getafe,17336.0,Estadio Manuel Martínez Valero,Guillermo Cuadra,...,3,1,La Liga,Matchweek 38,3-4-3,3-5-2,59.0,41.0,Pere Milla,Djené


In [7]:
# Do not need score and comp and round
scores_merge_table.drop(['score', 'comp', 'round'], axis = 1, inplace =True)
scores_merge_table

,_week_,_day_,_date_,_time_,home,away,attendance,venue,referee,xg_home,xg_away,homegoal,awaygoal,homeformation,awayformation,homepossession,awaypossession,homecaptain,awaycaptain
0,1,Fri,2017-08-18,20:15:00,Leganés,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,1.4,1.2,1,0,4-2-3-1,4-4-2,54.0,46.0,Martín Mantovani,Manu García
1,1,Fri,2017-08-18,22:15:00,Valencia,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,1.9,0.3,1,0,4-4-2,4-5-1,48.0,52.0,Daniel Parejo,Jonathan Viera
2,1,Sat,2017-08-19,18:15:00,Celta Vigo,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,2.4,2,3,4-3-3,4-3-3,53.0,47.0,Hugo Mallo,Xabi Prieto
3,1,Sat,2017-08-19,20:15:00,Girona,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,2.1,0.8,2,2,3-4-3,4-4-2,53.0,47.0,Álex Granell,Gabi
4,1,Sat,2017-08-19,22:15:00,Sevilla,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,1.1,1,1,4-1-4-1,4-4-2,62.0,38.0,Jesús Navas,Gerard Moreno
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,38,Sun,2022-05-22,22:00:00,Sevilla,Athletic Club,31305.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,1.4,0.7,1,0,4-3-3,4-4-2,65.0,35.0,Jesús Navas,Iker Muniain
1896,38,Sun,2022-05-22,22:00:00,Real Sociedad,Atlético Madrid,23586.0,Estadio Municipal de Anoeta,Jesús Gil,2.3,0.8,1,2,4-4-2◆,3-5-2,61.0,39.0,Asier Illarramendi,Koke
1897,38,Sun,2022-05-22,20:00:00,Granada,Espanyol,17951.0,Estadio Nuevo Los Cármenes,Alejandro Hernández,2.0,0.4,0,0,4-3-3,4-4-2,40.0,60.0,Víctor Díaz,David López
1898,38,Sun,2022-05-22,17:30:00,Elche,Getafe,17336.0,Estadio Manuel Martínez Valero,Guillermo Cuadra,2.5,1.3,3,1,3-4-3,3-5-2,59.0,41.0,Pere Milla,Djené


In [8]:
shooting_merge.head()

,_date_,_time_,comp,round,_day_,home,away,homegoal,xg_home,homeshots,...,home_penkicks,home_penkicksattempt,awaygoal,xg_away,awayshots,awayshots_on_target,away_distance,away_freekicks,away_penkicks,away_penkicksattempt
0,2017-08-18,20:15:00,La Liga,Matchweek 1,Fri,Leganés,Alavés,1,1.4,14,...,0,0,0,1.2,8,3,21.7,0,0,1
1,2017-08-18,22:15:00,La Liga,Matchweek 1,Fri,Valencia,Las Palmas,1,1.9,22,...,0,0,0,0.3,5,4,22.9,1,0,0
2,2017-08-19,22:15:00,La Liga,Matchweek 1,Sat,Sevilla,Espanyol,1,2.1,9,...,0,0,1,1.1,9,6,16.1,0,0,0
3,2017-08-19,20:15:00,La Liga,Matchweek 1,Sat,Girona,Atlético Madrid,2,2.1,14,...,0,0,2,0.8,9,3,18.0,0,0,0
4,2017-08-19,18:15:00,La Liga,Matchweek 1,Sat,Celta Vigo,Real Sociedad,2,1.6,16,...,0,0,3,2.4,13,5,17.5,1,1,1


In [9]:
scores_shooting_merge = scores_merge_table.merge(shooting_merge, on = ['_date_', '_time_', '_day_', 'home', 'away', 'homegoal', 'awaygoal', 'xg_home', 'xg_away'])
scores_shooting_merge

,_week_,_day_,_date_,_time_,home,away,attendance,venue,referee,xg_home,...,home_distance,home_freekicks,home_penkicks,home_penkicksattempt,awayshots,awayshots_on_target,away_distance,away_freekicks,away_penkicks,away_penkicksattempt
0,1,Fri,2017-08-18,20:15:00,Leganés,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,1.4,...,20.8,2,0,0,8,3,21.7,0,0,1
1,1,Fri,2017-08-18,22:15:00,Valencia,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,1.9,...,15.9,2,0,0,5,4,22.9,1,0,0
2,1,Sat,2017-08-19,18:15:00,Celta Vigo,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,...,16.9,1,0,0,13,5,17.5,1,1,1
3,1,Sat,2017-08-19,20:15:00,Girona,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,2.1,...,16.8,0,0,0,9,3,18.0,0,0,0
4,1,Sat,2017-08-19,22:15:00,Sevilla,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,...,11.6,0,0,0,9,6,16.1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,38,Sun,2022-05-22,22:00:00,Sevilla,Athletic Club,31305.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,1.4,...,17.9,0,0,0,10,1,17.2,0,0,0
1896,38,Sun,2022-05-22,22:00:00,Real Sociedad,Atlético Madrid,23586.0,Estadio Municipal de Anoeta,Jesús Gil,2.3,...,16.1,2,0,0,6,5,17.5,0,0,0
1897,38,Sun,2022-05-22,20:00:00,Granada,Espanyol,17951.0,Estadio Nuevo Los Cármenes,Alejandro Hernández,2.0,...,17.9,1,0,1,10,5,19.5,0,0,0
1898,38,Sun,2022-05-22,17:30:00,Elche,Getafe,17336.0,Estadio Manuel Martínez Valero,Guillermo Cuadra,2.5,...,16.7,0,0,0,8,2,20.3,1,0,0


In [10]:
scores_shooting_merge.columns

Index(['_week_', '_day_', '_date_', '_time_', 'home', 'away', 'attendance',
       'venue', 'referee', 'xg_home', 'xg_away', 'homegoal', 'awaygoal',
       'homeformation', 'awayformation', 'homepossession', 'awaypossession',
       'homecaptain', 'awaycaptain', 'comp', 'round', 'homeshots',
       'homeshots_on_target', 'home_distance', 'home_freekicks',
       'home_penkicks', 'home_penkicksattempt', 'awayshots',
       'awayshots_on_target', 'away_distance', 'away_freekicks',
       'away_penkicks', 'away_penkicksattempt'],
      dtype='object')

In [11]:
scores_shooting_merge.drop(['comp', 'round'], axis = 1, inplace = True)

In [12]:
scores_shooting_merge

,_week_,_day_,_date_,_time_,home,away,attendance,venue,referee,xg_home,...,home_distance,home_freekicks,home_penkicks,home_penkicksattempt,awayshots,awayshots_on_target,away_distance,away_freekicks,away_penkicks,away_penkicksattempt
0,1,Fri,2017-08-18,20:15:00,Leganés,Alavés,9231.0,Estadio Municipal de Butarque,José Munuera,1.4,...,20.8,2,0,0,8,3,21.7,0,0,1
1,1,Fri,2017-08-18,22:15:00,Valencia,Las Palmas,35971.0,Estadio de Mestalla,Jesús Gil,1.9,...,15.9,2,0,0,5,4,22.9,1,0,0
2,1,Sat,2017-08-19,18:15:00,Celta Vigo,Real Sociedad,16961.0,Estadio de Balaídos,Antonio Matéu Lahoz,1.6,...,16.9,1,0,0,13,5,17.5,1,1,1
3,1,Sat,2017-08-19,20:15:00,Girona,Atlético Madrid,11511.0,Estadi Municipal de Montilivi,Juan Martínez,2.1,...,16.8,0,0,0,9,3,18.0,0,0,0
4,1,Sat,2017-08-19,22:15:00,Sevilla,Espanyol,30487.0,Estadio Ramón Sánchez Pizjuán,Alejandro Hernández,2.1,...,11.6,0,0,0,9,6,16.1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,38,Sun,2022-05-22,22:00:00,Sevilla,Athletic Club,31305.0,Estadio Ramón Sánchez Pizjuán,Carlos del Cerro,1.4,...,17.9,0,0,0,10,1,17.2,0,0,0
1896,38,Sun,2022-05-22,22:00:00,Real Sociedad,Atlético Madrid,23586.0,Estadio Municipal de Anoeta,Jesús Gil,2.3,...,16.1,2,0,0,6,5,17.5,0,0,0
1897,38,Sun,2022-05-22,20:00:00,Granada,Espanyol,17951.0,Estadio Nuevo Los Cármenes,Alejandro Hernández,2.0,...,17.9,1,0,1,10,5,19.5,0,0,0
1898,38,Sun,2022-05-22,17:30:00,Elche,Getafe,17336.0,Estadio Manuel Martínez Valero,Guillermo Cuadra,2.5,...,16.7,0,0,0,8,2,20.3,1,0,0


In [13]:
possession_merge.head()

,_date_,_time_,comp,round,_day_,home,away,homegoal,homepossession,home_touches,...,away_carries_tot_dist,away_carries_prog_dist,away_carries_prog,away_carries_one_third,away_carries_cpa,away_carries_miss,away_carries_dis,away_receiving_target,away_receiving_rec,away_receiving_prog
0,2017-08-18,20:15:00,La Liga,Matchweek 1,Fri,Leganés,Alavés,1,54,566,...,1542,758,21.0,5.0,1.0,15,15,309,250,250
1,2017-08-18,22:15:00,La Liga,Matchweek 1,Fri,Valencia,Las Palmas,1,48,597,...,2691,1439,52.0,15.0,2.0,15,12,499,451,451
2,2017-08-19,22:15:00,La Liga,Matchweek 1,Sat,Sevilla,Espanyol,1,62,787,...,1531,755,32.0,17.0,3.0,5,12,401,297,297
3,2017-08-19,18:15:00,La Liga,Matchweek 1,Sat,Celta Vigo,Real Sociedad,2,53,591,...,1896,1013,30.0,10.0,3.0,14,7,419,352,352
4,2017-08-19,20:15:00,La Liga,Matchweek 1,Sat,Girona,Atlético Madrid,2,53,619,...,1892,1169,44.0,15.0,3.0,5,8,406,347,347


In [14]:
possession_merge.columns

Index(['_date_', '_time_', 'comp', 'round', '_day_', 'home', 'away',
       'homegoal', 'homepossession', 'home_touches', 'home_touches_def_pen',
       'home_touches_def_third', 'home_touches_mid_third',
       'home_touches_att_third', 'home_touches_att_pen', 'home_touches_live',
       'home_dribbles_success', 'home_dribbles_att',
       'home_dribbles_players_number', 'home_dribbles_megs', 'home_carries',
       'home_carries_tot_dist', 'home_carries_prog_dist', 'home_carries_prog',
       'home_carries_one_third', 'home_carries_cpa', 'home_carries_miss',
       'home_carries_dis', 'home_receiving_target', 'home_receiving_rec',
       'home_receiving_prog', 'awaygoal', 'awaypossession', 'away_touches',
       'away_touches_def_pen', 'away_touches_def_third',
       'away_touches_mid_third', 'away_touches_att_third',
       'away_touches_att_pen', 'away_touches_live', 'away_dribbles_success',
       'away_dribbles_att', 'away_dribbles_players_number',
       'away_dribbles_megs', 'a

In [15]:
laligafinal_merge = scores_shooting_merge.merge(possession_merge, on = ['_date_', '_time_', '_day_', 'home', 'away', 'homegoal', 'awaygoal', 'homepossession', 'awaypossession'])

In [16]:
def create_result_col(row):
    if row.homegoal > row.awaygoal:
        return "HW"
    elif row.awaygoal > row.homegoal:
        return "AW"
    else:
        return "D"

In [17]:
laligafinal_merge['match_result'] = laligafinal_merge.apply(create_result_col, axis =1)

In [18]:
laligafinal_merge.columns

Index(['_week_', '_day_', '_date_', '_time_', 'home', 'away', 'attendance',
       'venue', 'referee', 'xg_home', 'xg_away', 'homegoal', 'awaygoal',
       'homeformation', 'awayformation', 'homepossession', 'awaypossession',
       'homecaptain', 'awaycaptain', 'homeshots', 'homeshots_on_target',
       'home_distance', 'home_freekicks', 'home_penkicks',
       'home_penkicksattempt', 'awayshots', 'awayshots_on_target',
       'away_distance', 'away_freekicks', 'away_penkicks',
       'away_penkicksattempt', 'comp', 'round', 'home_touches',
       'home_touches_def_pen', 'home_touches_def_third',
       'home_touches_mid_third', 'home_touches_att_third',
       'home_touches_att_pen', 'home_touches_live', 'home_dribbles_success',
       'home_dribbles_att', 'home_dribbles_players_number',
       'home_dribbles_megs', 'home_carries', 'home_carries_tot_dist',
       'home_carries_prog_dist', 'home_carries_prog', 'home_carries_one_third',
       'home_carries_cpa', 'home_carries_miss', '

In [19]:
laligafinal_merge[['_date_' , '_time_', 'home', 'away', 'homegoal', 'awaygoal', 'match_result']]

,_date_,_time_,home,away,homegoal,awaygoal,match_result
0,2017-08-18,20:15:00,Leganés,Alavés,1,0,HW
1,2017-08-18,22:15:00,Valencia,Las Palmas,1,0,HW
2,2017-08-19,18:15:00,Celta Vigo,Real Sociedad,2,3,AW
3,2017-08-19,20:15:00,Girona,Atlético Madrid,2,2,D
4,2017-08-19,22:15:00,Sevilla,Espanyol,1,1,D
...,...,...,...,...,...,...,...
1895,2022-05-22,22:00:00,Sevilla,Athletic Club,1,0,HW
1896,2022-05-22,22:00:00,Real Sociedad,Atlético Madrid,1,2,AW
1897,2022-05-22,20:00:00,Granada,Espanyol,0,0,D
1898,2022-05-22,17:30:00,Elche,Getafe,3,1,HW


In [20]:
laligafinal_merge.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1900 entries, 0 to 1899
Data columns (total 78 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   _week_                        1900 non-null   int64  
 1   _day_                         1900 non-null   object 
 2   _date_                        1900 non-null   object 
 3   _time_                        1900 non-null   object 
 4   home                          1900 non-null   object 
 5   away                          1900 non-null   object 
 6   attendance                    1412 non-null   float64
 7   venue                         1900 non-null   object 
 8   referee                       1900 non-null   object 
 9   xg_home                       1900 non-null   float64
 10  xg_away                       1900 non-null   float64
 11  homegoal                      1900 non-null   int64  
 12  awaygoal                      1900 non-null   int64  
 13  hom

In [21]:
# laligafinal_merge.to_pickle(os.path.join(os.path.realpath('../data/preprocessed/'), 'match_merged.pkl'))

In [22]:
laligafinal_merge.to_csv(os.path.join(os.path.realpath('../data/preprocessed/'), 'match_merged.csv'), index = False)

In [24]:
for i in laligafinal_merge.columns:
    print(i)

_week_
_day_
_date_
_time_
home
away
attendance
venue
referee
xg_home
xg_away
homegoal
awaygoal
homeformation
awayformation
homepossession
awaypossession
homecaptain
awaycaptain
homeshots
homeshots_on_target
home_distance
home_freekicks
home_penkicks
home_penkicksattempt
awayshots
awayshots_on_target
away_distance
away_freekicks
away_penkicks
away_penkicksattempt
comp
round
home_touches
home_touches_def_pen
home_touches_def_third
home_touches_mid_third
home_touches_att_third
home_touches_att_pen
home_touches_live
home_dribbles_success
home_dribbles_att
home_dribbles_players_number
home_dribbles_megs
home_carries
home_carries_tot_dist
home_carries_prog_dist
home_carries_prog
home_carries_one_third
home_carries_cpa
home_carries_miss
home_carries_dis
home_receiving_target
home_receiving_rec
home_receiving_prog
away_touches
away_touches_def_pen
away_touches_def_third
away_touches_mid_third
away_touches_att_third
away_touches_att_pen
away_touches_live
away_dribbles_success
away_dribbles_att